In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

ignore_cols = ["id", "date", "zipcode"]
for col in ignore_cols:
    if col in train.columns:
        train = train.drop(columns=col)
    if col in test.columns:
        test = test.drop(columns=col)

train["price"] = train["price"] / 1000
test["price"] = test["price"] / 1000

X_train = train.drop("price", axis=1)
y_train = train["price"]
X_test = test.drop("price", axis=1)
y_test = test["price"]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [5]:
######## Problem 2:  Linear regression #######

# 1. and 2.

model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Predictions
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# Metrics
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Train MSE:", train_mse)
print("Train R2:", train_r2)
print("Test MSE:", test_mse)
print("Test R2:", test_r2)
 
coefficients = pd.DataFrame({
    "Feature": X_train.columns,
    "Coefficient": model.coef_
})

print(coefficients.sort_values(by="Coefficient", ascending=False))


Train MSE: 31415.747916100863
Train R2: 0.7271450489303788
Test MSE: 58834.673978213985
Test R2: 0.6471195893437872
          Feature  Coefficient
9           grade    92.511076
14            lat    78.129852
6      waterfront    64.230911
3     sqft_living    57.161582
10     sqft_above    48.439051
7            view    47.610288
16  sqft_living15    45.479128
11  sqft_basement    27.688812
2       bathrooms    18.456913
13   yr_renovated    17.341926
8       condition    12.647609
4        sqft_lot    11.127338
0      Unnamed: 0     8.456024
5          floors     8.151038
15           long    -1.437669
1        bedrooms   -12.807339
17     sqft_lot15   -12.906560
12       yr_built   -68.043173


In [29]:
######## Problem 2 continued #######

""" 3. The model explains around 73% of the variance in the training data and 65% in the testing data, therefore it fits well but not perfectly of course. The features that contribute the most are grade, 
latitude, waterfront, and sqft_living, which have the largest coefficients and have a big influence on housing price. The training MSE is lower than the testing MSE, 
which means the data is a bit overfitted but is still a reasonable generalization. """

' 3. The model explains around 73% of the variance in the training data and 65% in the testing data, therefore it fits well but not perfectly of course. The features that contribute the most are grade, \nlatitude, waterfront, and sqft_living, which have the largest coefficients and have a big influence on housing price. The training MSE is lower than the testing MSE, \nwhich means the data is a bit overfitted but is still a reasonable generalization. '

In [7]:
##### Problem 3:  Implementing closed-form solution for linear regression #####

X_train_bias = np.c_[np.ones(X_train_scaled.shape[0]), X_train_scaled]
X_test_bias = np.c_[np.ones(X_test_scaled.shape[0]), X_test_scaled]

theta = np.linalg.inv(X_train_bias.T @ X_train_bias) @ X_train_bias.T @ y_train.values

y_train_pred_cf = X_train_bias @ theta
y_test_pred_cf = X_test_bias @ theta

print("Train MSE:", mean_squared_error(y_train, y_train_pred_cf))
print("Train R2:", r2_score(y_train, y_train_pred_cf))
print("Test MSE:", mean_squared_error(y_test, y_test_pred_cf))
print("Test R2:", r2_score(y_test, y_test_pred_cf))


Train MSE: 34334.80751744762
Train R2: 0.7017921632750106
Test MSE: 59948.01817993596
Test R2: 0.6404419393707315


In [ ]:
# Problem 3 Discussion

""" Compared to the sk learn model I chose to use from Problem 2, the results are very similar, with only small differences in MSE and R squared,
which means this function is accurate """

In [19]:
#### Problem 4: Polynomial Regression ####

def polynomial_features(X, degree):
    X_poly = np.ones((X.shape[0], 1))
    for d in range(1, degree + 1):
        X_poly = np.c_[X_poly, X**d]
    return X_poly

feature = "sqft_living"

X_train_sqft = train[[feature]].values
X_test_sqft = test[[feature]].values

scaler_sqft = StandardScaler()
X_train_sqft = scaler_sqft.fit_transform(X_train_sqft)
X_test_sqft = scaler_sqft.transform(X_test_sqft)

results = []

for p in [1,2,3,5]:
    X_train_poly = polynomial_features(X_train_sqft, p)
    X_test_poly = polynomial_features(X_test_sqft, p)

    theta_poly = np.linalg.inv(X_train_poly.T @ X_train_poly) @ X_train_poly.T @ y_train.values

    y_train_pred = X_train_poly @ theta_poly
    y_test_pred = X_test_poly @ theta_poly

    results.append([
        p,
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred),
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)
    ])

results_df = pd.DataFrame(results, columns=["Degree", "Train MSE", "Test MSE", "Train R2", "Test R2"])
results_df



,Degree,Train MSE,Test MSE,Train R2,Test R2
0,1,57947.526161,88575.978543,0.496709,0.468736
1,2,54822.665116,71791.679479,0.523849,0.569406
2,3,53785.194716,99833.483763,0.532860,0.401216
3,5,52626.111955,570616.914820,0.542927,-2.422464


In [31]:
### Problem 4 Cont ####

""" As the polynomial degree increases, the training MSE decreases and training R2 increases slightly, which is expected since more complex models fit the training data better. 
But, the testing performance improves only up to degree 2 and then worsens significantly for higher degrees. Like for example degree 5, the testing R2 becomes negative.
This shows that while adding polynomial terms increases model flexibility, higher-degree polynomials might not generalize well. """


' As the polynomial degree increases, the training MSE decreases and training R2 increases slightly, which is expected since more complex models fit the training data better. \nBut, the testing performance improves only up to degree 2 and then worsens significantly for higher degrees. Like for example degree 5, the testing R2 becomes negative.\nThis shows that while adding polynomial terms increases model flexibility, higher-degree polynomials might not generalize well. '

In [39]:
#### Problem 5:  Gradient descent #####

def gradient_descent(X, y, alpha, iterations):
    m = len(y)
    theta = np.zeros(X.shape[1])

    for _ in range(iterations):
        predictions = X @ theta
        gradient = (1/m) * X.T @ (predictions - y)
        theta = theta - alpha * gradient

    return theta

results = []


X_train_bias = np.c_[np.ones(X_train_scaled.shape[0]), X_train_scaled]
X_test_bias = np.c_[np.ones(X_test_scaled.shape[0]), X_test_scaled]


for alpha in [0.01, 0.1, 0.5]:
    for it in [10, 50, 100]:
        theta_gd = gradient_descent(X_train_bias, y_train.values, alpha, it)

        y_train_pred = X_train_bias @ theta_gd
        y_test_pred = X_test_bias @ theta_gd

        results.append([
            alpha,
            it,
            mean_squared_error(y_train, y_train_pred),
            mean_squared_error(y_test, y_test_pred),
            r2_score(y_train, y_train_pred),
            r2_score(y_test, y_test_pred)
        ])

        print("Alpha:", alpha, "Iterations:", it)
        print("Theta:", theta_gd)

results_df = pd.DataFrame(results, 
    columns=["Alpha", "Iterations", "Train MSE", "Test MSE", "Train R2", "Test R2"])

results_df




Alpha: 0.01 Iterations: 10
Theta: [ 4.97609866e+01 -5.71355940e-01  7.88230818e+00  1.28885855e+01
  1.94697369e+01  3.65596272e+00  6.19596850e+00  9.65192771e+00
  1.30928688e+01  2.52788366e+00  1.79913003e+01  1.58321121e+01
  1.05991846e+01 -9.86859516e-01  4.55126922e+00  1.13477059e+01
  1.14764514e-02  1.78147002e+01  4.02621859e+00]
Alpha: 0.01 Iterations: 50
Theta: [ 2.05560702e+02 -3.15750389e-02  1.25163535e+01  2.59212818e+01
  4.79069491e+01  5.45299794e+00  1.16687762e+01  3.27540471e+01
  3.92741651e+01  1.03813881e+01  4.62406985e+01  3.72421944e+01
  2.90836406e+01 -1.65007790e+01  1.58020462e+01  4.09624652e+01
 -7.61827526e+00  4.41320794e+01  5.68597809e+00]
Alpha: 0.01 Iterations: 100
Theta: [329.92617388   2.14260858   6.03789753  23.83369358  54.60665253
   3.66935537  11.13059106  46.01543057  49.51375953  14.50279947
  56.5436346   42.4780124   33.10264224 -31.88671648  21.22889322
  59.96091374 -13.09620479  51.20633612   2.71040894]
Alpha: 0.1 Iterations: 10

,Alpha,Iterations,Train MSE,Test MSE,Train R2,Test R2
0,0.01,10,2.947967e+05,3.524490e+05,-1.560395e+00,-1.113929e+00
1,0.01,50,1.382985e+05,1.704509e+05,-2.011631e-01,-2.233580e-02
2,0.01,100,7.009438e+04,9.475122e+04,3.912098e-01,4.316983e-01
3,0.10,10,6.647361e+04,9.087301e+04,4.226573e-01,4.549591e-01
4,0.10,50,3.151072e+04,5.892963e+04,7.263202e-01,6.465501e-01
5,0.10,100,3.142750e+04,5.889174e+04,7.270429e-01,6.467773e-01
6,0.50,10,6.163610e+08,6.888098e+08,-5.352276e+03,-4.130365e+03
7,0.50,50,1.708464e+25,1.904481e+25,-1.483851e+20,-1.142275e+20
8,0.50,100,6.110787e+45,6.811893e+45,-5.307397e+40,-4.085658e+40


In [55]:
""" When α = 0.01, the convergence is slow and performance improves gradually but does not reach the optimal solution by 100 iterations. When α = 0.1, it converges and reaches almost an optimal MSE and R² values after 50 iterations. With α = 0.5, the algorithm diverges, causing larger error and worse R² values. This shows that an appropriate learning rate is necessary for stable convergence to the optimal solution.

"""


' When α = 0.01, the convergence is slow and performance improves gradually but does not reach the optimal solution by 100 iterations. When α = 0.1, it converges and reaches almost an optimal MSE and R² values after 50 iterations. With α = 0.5, the algorithm diverges, causing larger error and worse R² values. This shows that an appropriate learning rate is necessary for stable convergence to the optimal solution.\n\n'

In [57]:
##### Problem 6: Ridge regularization #####

def ridge_gradient_descent(X, y, alpha, iterations, lam):
    m = len(y)
    theta = np.zeros(X.shape[1])

    for _ in range(iterations):
        predictions = X @ theta
        gradient = (1/m) * (X.T @ (predictions - y))
        
        # Regularization term (exclude bias)
        reg = 2 * lam * theta
        reg[0] = 0
        
        theta = theta - alpha * (gradient + reg)

    return theta


np.random.seed(0)

N = 1000
X = np.random.uniform(-2, 2, N)
eps = np.random.normal(0, np.sqrt(2), N)
Y = 1 + 2*X + eps

X_design = np.column_stack((np.ones(N), X))

def ridge_closed_form(X, y, lam):
    I = np.eye(X.shape[1])
    I[0,0] = 0  # don't regularize bias
    return np.linalg.inv(X.T @ X + lam * I) @ X.T @ y

theta_ols = ridge_closed_form(X_design, Y, 0)
y_pred_ols = X_design @ theta_ols

print("OLS slope:", theta_ols[1])
print("OLS MSE:", mean_squared_error(Y, y_pred_ols))
print("OLS R2:", r2_score(Y, y_pred_ols))
print()

for lam in [1,10,100,1000,10000]:
    theta_ridge = ridge_closed_form(X_design, Y, lam)
    y_pred = X_design @ theta_ridge
    
    print("Lambda:", lam)
    print("Slope:", theta_ridge[1])
    print("MSE:", mean_squared_error(Y, y_pred))
    print("R2:", r2_score(Y, y_pred))
    print()




OLS slope: 1.9656920054163671
OLS MSE: 1.8653740489434376
OLS R2: 0.7367593726211263

Lambda: 1
Slope: 1.964238266463562
MSE: 1.865376904433118
R2: 0.7367589696558897

Lambda: 10
Slope: 1.9512507370045533
MSE: 1.865655834299756
R2: 0.7367196072164208

Lambda: 100
Slope: 1.8302356836661557
MSE: 1.8901657483079775
R2: 0.7332607807444963

Lambda: 1000
Slope: 1.1296410740692593
MSE: 2.809811521006989
R2: 0.6034808418047767

Lambda: 10000
Slope: 0.23398221709987252
MSE: 5.917267005381879
R2: 0.164958320425167



In [53]:
### Observations: As λ increases, the slope goes toward 0, the coefficients become smaller, training MSE increases, R² decreases.